In [1]:
import cv2
import numpy as np
import dlib
import os
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from skimage.feature import hog
from skimage import exposure

In [6]:
dir_train = r'.\preprocessed_dataset\Women\train'
dir_test = r'.\preprocessed_dataset\Women\test'
dir_model = r'.\model'


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dir_model + "\\" + "shape_predictor_68_face_landmarks.dat")

In [15]:
def edge_segmentation(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Feature extraction (dummy example)

def extract_features(image):
    # Resize the image to 128x128
    image_resized = cv2.resize(image, (128, 128))
    # Compute HOG descriptors
    features, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(4, 4), block_norm="L2", visualize=True)
    return features, hog_image


In [16]:
def feature_engineering(path):
    labels = []
    hog_features = []
    shape_types = ['Heart', 'Oblong', 'Oval', 'Round', 'Square']

    for shape in shape_types:
        print(f'Loading {shape} images...')
        traindir = os.listdir(path + "\\" + shape)
        for imgName in traindir:
            print(f'Processing {imgName}...', end=" ")
            img_path = os.path.join(path, shape, imgName)
            img = cv2.imread(img_path)
            # Edge segmentation + Kmeans
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # img = cv2.GaussianBlur(img, (5, 5), 0)
            img = edge_segmentation(img)
            
            # Hog feature
            hog_feature , _ = extract_features(img)
            hog_features.append(hog_feature)
            labels.append(shape_types.index(shape))
        print("")
        
    labels = np.array(labels).reshape(len(labels), 1)
    hog_features = np.array(hog_features)

    data = np.hstack((hog_features, labels))
    return data

In [17]:
data_train = feature_engineering(dir_train)
data_test = feature_engineering(dir_test)

Loading Heart images...
Processing heart (1).jpg... Processing heart (10).jpg... Processing heart (100).jpg... Processing heart (1000).jpg... Processing heart (103).jpg... Processing heart (104).jpg... Processing heart (105).jpg... Processing heart (106).jpg... Processing heart (107).jpg... Processing heart (108).jpg... Processing heart (109).jpg... Processing heart (11).jpg... Processing heart (110).jpg... Processing heart (111).jpg... Processing heart (112).jpg... Processing heart (113).jpg... Processing heart (114).jpg... Processing heart (115).jpg... Processing heart (116).jpg... Processing heart (117).jpg... Processing heart (118).jpg... Processing heart (119).jpg... Processing heart (12).jpg... Processing heart (120).jpg... Processing heart (122).jpg... Processing heart (124).jpg... Processing heart (125).jpg... Processing heart (126).jpg... Processing heart (127).jpg... Processing heart (128).jpg... Processing heart (129).jpg... Processing heart (130).jpg... Processing heart (13

In [18]:
np.random.shuffle(data_train)
x_train = data_train[:, :-1]
y_train = data_train[:, -1]
np.random.shuffle(data_test)
x_test = data_test[:, :-1]
y_test = data_test[:, -1]

## ĐÂY LÀ PHẦN BUILD MODEL

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

para = {
    'C': [1, 2, 5],
    'gamma': [0.1, 0.01, 1, "scale"]
}
model = SVC(kernel = 'rbf', random_state = 42)
search = GridSearchCV(model, para)
search.fit(x_train[:300], y_train[:300])

GridSearchCV(estimator=SVC(random_state=42),
             param_grid={'C': [1, 2, 5], 'gamma': [0.1, 0.01, 1, 'scale']})

In [20]:
best_params = search.best_params_
best_score = search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'C': 5, 'gamma': 0.01}
Best cross-validation score: 0.3466666666666667


In [13]:
from sklearn.svm import SVC

model = SVC(kernel = 'rbf', C = 2, gamma=0.01)
model.fit(x_train, y_train)

SVC(C=2, gamma=0.01)

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[116  27  24  20  13]
 [ 29 132  20  10   7]
 [ 39  26  88  37   8]
 [ 10  12  26 113  36]
 [  8  13  13  30 136]]


0.5891238670694864

### Save model after training (if training on tensorflow, please save model using this [link](https://www.geeksforgeeks.org/save-and-load-models-in-tensorflow/))

In [ ]:
%pip install joblib

In [ ]:
import pickle

# DON'T RUN IF YOU ALREADY HAVE THE MODEL FILES
file_name = './model/ml-model.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(model, file)